## How To: Retrieve Trades for a Portfolio and Trade Date

The Trades API retrieves trades by the Aladdin trade id, external trade ID, portfolio group, trade date, settlement date, or a combination of these parameters.

The following demo retrieves trades by portfolio (group) and trade date, and stores the results in a DataFrame.

In [ ]:
#import libraries for API Calls / Data Analysis
import uuid,datetime,requests,pprint,math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#set Client Environment (from url), Aladdin Username & Password, and APIkey here
client_env = ''
username = ''
pwd = ''
APIkey= ''

today = datetime.date.today().strftime('%Y-%m-%d')
        
#function to generate unique headers when called
def generate_headers():
    headers = {
        'VND.com.blackrock.API-Key': APIkey,
        'VND.com.blackrock.Origin-Timestamp': str(datetime.datetime.utcnow().replace(microsecond=0).astimezone().isoformat()),
        'VND.com.blackrock.Request-ID': str(uuid.uuid1())
    }
    return headers

### Prepare API Parameters and Set URL

In [ ]:
#set request parameters and URL
fund = 'TST-TRD1'

trades_payload = {
    'portGroup': fund,
    'tradeDate':today
}

url = f'https://{client_env}.blackrock.com/api/trading/trades/v1/trade'

### Send API Request and Store Response

In [ ]:
r = requests.get(url,auth=(username,pwd),params=trades_payload,headers=generate_headers()).json()
r

### Convert Response to DataFrame

In [ ]:
#create allTrades DataFrame, loop through each Portfolio and add all Invnums to DataFrame
allTrades = pd.DataFrame()
for portfolio in r['tradeDataByPortfolioId']:
    allTrades = allTrades.append(pd.DataFrame.from_dict(r['tradeDataByPortfolioId'][portfolio]['tradeRecordByInvnum'],orient='index').reset_index())

#order columns in a logical manner 
allTrades = allTrades[['portfolioTicker','index','assetId','tradeDate','settleDate','tradeTranType','tradePrice','tradeQuantity','cptyId','cptyName','multiFundId','trader']].sort_values(by=['portfolioTicker','index']).reset_index(drop=True)
allTrades.rename(columns={'index':'invnum'}, inplace=True)
 
#print top 5 rows
allTrades.head()